# Cost of Plutocracy

This article aims to analyze some of the biggest DAOs in the crypto space and see how many of them are influenced by token holders which own a disproportionate amount of tokens compared to most voters.

In [1]:
# sets up the pynb environment
import os
import sys

import pandas as pd
import numpy as np


module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from libs.data_processing.filters import get_quartile_by_vp
from libs.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
)


The data used for this article came from data sources that specialize in providing data on DAO governance, such as DeepDAO and Snapshot. Coingecko was also used to find market data for DAO native tokens.

Using this data, two spreadsheets were compiled which act as the local database for this analysis. Each spreadsheet contains each voter's choice and voting power for the last fifty proposals each DAO opened. **One spreadsheet filters out "whales"** which, in the context of this analysis, are voters of each proposal that have voting power **at or above the 95th percentile of voting power for that proposal**.

In [2]:
plutocracy_report_data = pd.read_excel(
    "../plutocracy_report.xlsx", sheet_name=None, engine="openpyxl"
)
plutocracy_report_data_filtered = pd.read_excel(
    "../plutocracy_report_filtered.xlsx", sheet_name=None, engine="openpyxl"
)

With this, we can begin asking ourselves some questions. Such as: **How many whales have voted in the last fifty proposals for each of the DAOs we analyzed?**

In [3]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [4]:
pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    index=[list(result.items())[0][0] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
        "avg vp for non-whales",
        "avg vp for whales",
        "avg cost of vote",
    ],
)


,# of whales,all voters,avg vp for non-whales,avg vp for whales,avg cost of vote
Uniswap,791,21085,1.675602,9.507515e+04,7.289716
ENS,321,9742,281.564683,5.561220e+04,29.874161
Lido,152,4384,115.907850,1.351838e+06,1.564882
Frax,2,161,112522.786545,1.436326e+07,8.416601
Curve Finance,14,235,3454.929793,1.352748e+06,3.338636
Decentraland,54,1182,251.487140,2.851501e+05,0.703804
Compound,55,1054,0.128663,5.490055e+03,47.541103
Radicle,18,395,26779.601995,9.430134e+05,12.567908


Here we can see that for the DAOs which we choose to analyze the top 5% of voters have a clear, disproportionate amount of voting power compared to the average voter.

We can also gain some insight into the economic might that these whales hold for each DAO at the time of voting by taking a look at the average cost of each vote for each DAO's native token at the time these proposals were active (in USD).

---

We move on to take a look at what DAO participation would look like without whale involvement. We do so by comparing the scores of each proposal and checking whether or not the outcome of the proposal was changed after filtering out whales.

We check whether or not a proposal's outcome has changed simply by checking if the largest vote choice score has changed after filtering out whales, more specifically, in python we do:
```python
has_changed_outcome = not unfiltered_winning_choice_index == filtered_winning_choice_index
```

In [5]:
pd.set_option("display.max_rows", 5)
score_diffrences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [6]:
score_diffrences_dfs = dict()

for score_diffrence in score_diffrences:
    for organization, data in score_diffrence.items():
        data: dict[str, list] = data
        items = data.items()
        score_diffrences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([proposal_id for proposal_id, _ in items]), name="Proposal ID"
            ),
            columns=[
                "type",
                "choices",
                "score_diffrences",
                "total_vp",
                "outcome_changed",
                "outcome"
            ],
        )
        score_diffrences_dfs[organization]["total_vp"] = score_diffrences_dfs[
            organization
        ]["total_vp"].apply("{:.9f}".format)
        score_diffrences_dfs[organization].index = score_diffrences_dfs[organization].index.str.slice(0, 9)


First, let's look at Uniswap

In [7]:
score_diffrences_dfs["Uniswap"].index = score_diffrences_dfs["Uniswap"].index.str.slice(0, 9)
score_diffrences_dfs["Uniswap"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0x374d7f9,single-choice,"['Yes', 'No']","[14155.085736103796, 30.84262236625821]",18625.532208988,False,Yes
0x15031b6,single-choice,"['For', 'Against', 'Abstain']","[15561440.496482328, 0.0, 0.0]",38105686.407799788,False,For
...,...,...,...,...,...,...
QmcpJ8Qm9,single-choice,"['Yes, Bring V3 to AVAX', 'No, V3 Should Stay ...","[9790.514303638305, 0]",10030.326504541,False,"Yes, Bring V3 to AVAX"
QmYeNLn1x,single-choice,"['No support', '25k - 50k (37.5k $UNI loan)', ...","[0.0, 0.0, 75.83513107330648, 135.62208, 0.0, ...",212333.479358470,False,800k+ (1M $UNI loan)


We can do some quick calculations using pandas to see that 12.5% of proposals had their outcome changed by removing whales.

In [8]:
score_diffrences_dfs["Uniswap"]["outcome_changed"].value_counts(normalize=True)

False    0.875
True     0.125
Name: outcome_changed, dtype: float64

One of the proposals which had their outcome changed after removing whales was the proposal to [add a Liquidity Mining Manager for the Optimism-Uniswap Liquidity Mining Program](https://snapshot.org/#/uniswap/proposal/0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c).

In this case, "Overnight Finance" would've been the chosen LMM if it weren't for whale intervention, instead "DeFiEdge" was chosen for this campaign. From the data, we can see that without whale intervention, DeFiEdge would've had ~2,708,025 fewer votes out of the total ~2,728,492 voting power on this proposal. That is ~99.3% of voting power allocated for this proposal directed to choosing "DeFiEdge".

In [9]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Uniswap"].loc["0xfd3d380"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,add Overnight Finance,add DeFiEdge,add none
Scores,14335.930887,2.710600e+06,3556.089173
Score Differeces,10054.585076,2.708025e+06,3412.851497


One highly contentious proposal whose outcome did not change after filtering out whales was [this temperature check to choose which Eth <> BNB bridge to use for Uniswap v3 governance](https://snapshot.org/#/uniswap/proposal/0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852). We see that even for the least popular choices, whales still contributed to the majority of the votes. We also observe that whales held the overwhelming majority of voting power for the runner-up, "LayerZero", with 99.9% of voting power allocated to this proposal being directed to this choice. By comparison, "Wormhole" received 65.1% of voting power for this proposal coming from whales.

In [10]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Uniswap"].loc["0x6b8df36"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Wormhole,LayerZero,Celer,deBridge
Scores,2.840391e+07,1.717334e+07,662.606204,3772.802474
Score Differeces,2.327546e+07,1.716363e+07,481.545728,3402.436704


Next up is ENS.

In [11]:
score_diffrences_dfs["ENS"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0x41b3509,single-choice,"['For', 'Against', 'Abstain']","[1667210.963067633, 0.0, 137.12454090065714]",1670185.073618348,False,For
0xd7eff78,single-choice,"['For', 'Against', 'Abstain']","[1666938.9315321625, 100.38682608595627, 141.9...",1670154.219044510,False,For
...,...,...,...,...,...,...
0xfe73d1b,approval,"['1. Transfer Treasury', '2. Transfer Registra...","[4573759.0782857565, 4573759.078285757, 457375...",4856109.997858677,False,1. Transfer Treasury
0xd810c4c,approval,"['Ratify Article I', 'Ratify Article II', 'Rat...","[170029.0968926698, 170029.0968926698, 170029....",16102806.718553530,False,Ratify Article I


In [12]:
score_diffrences_dfs["ENS"]["outcome_changed"].value_counts(normalize=True)

False    0.964286
True     0.035714
Name: outcome_changed, dtype: float64

We see that a very small percentage of proposal outcomes were changed after filtering out whales (3.5%). However, if we take look at score differences as they compare to each score, we see that they were almost all determined by whales.

In [13]:
propsal_choices = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["ENS"].loc["0x41b3509"]["score_diffrences"]
proposal_scores = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,For,Against,Abstain
Scores,1.669865e+06,125.702762,194.817417
Score Differeces,1.667211e+06,0.000000,137.124541


[This proposal](https://snapshot.org/#/ens.eth/proposal/0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7) once passed will approve the funding of various grants ENS has committed to funding as detailed by the proposal, **the amount of capital to be committed to this proposal is 250k USDC and 50 ETH**. 99.9% of voting power allocated towards this proposal was committed to voting for its passing, 0.0001% was allocated to abstaining and even a smaller percentage was allocated to voting against its passing.

Meaning that ~0.1% of the vote would've still been allocated towards voting for the proposal to pass even without whale intervention, which I think many would see as fine since the funding of public goods is always important.

Another proposal on ENS which had a large amount of voting power committed by whales is [this proposal](https://snapshot.org/#/ens.eth/proposal/0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b) to commit **935k USDC and 254 ETH** to the ENS Ecosystem Working group which is responsible for *growing and improving the ENS Ecosystem by funding people and projects that are ENS-specific or ENS-centric*.

In [14]:
propsal_choices = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["ENS"].loc["0x5788bf0"]["score_diffrences"]
proposal_scores = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,For,Against,Abstain
Scores,1.449464e+06,208582.378002,13640.353424
Score Differeces,1.446509e+06,208348.531592,13519.449465


Similar to the previous proposal we looked at, this one was highly influenced by whales (~86.5% of voting power committed to this proposal coming from whales that voted for its passing, ~12.5% from whales that voted against it, and ~0.8% of whales voted to abstain).

Yet, even after removing whales, the proposal still passes. After doing some quick math, we can see the remaining voting power for this proposal that voted for its passing equates to about ~0.17% out of the remaining 0.2% of voting power after removing whales.

Still, I am not at all saying that the outcome of this proposal is controversial. An organisation's funding initiatives that aid in the development of its product should not be a controversial topic for its members. I suppose the only thing that would be contested is the amount that is committed to this proposal.

---
Next lets take a look at everyone's favourite liquid staking platform: Lido.

In [15]:
score_diffrences_dfs["Lido"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0xc00b482,single-choice,"['For', 'Against']","[9121489.370392222, 69.40333267123832]",9124945.808187168,False,For
0x3436b98,single-choice,"['For', 'Against']","[55124323.664074376, 2048432.8880345668]",57176658.163439780,False,For
...,...,...,...,...,...,...
0xa2cd540,single-choice,"['For', 'Against']","[64857751.02380797, 0.0]",64866671.314562209,False,For
0xdeb8cd8,single-choice,"['For', 'Against']","[23322855.94982495, 90633.0303001409]",23470186.408249840,False,For


In [16]:
score_diffrences_dfs["Lido"]["outcome_changed"].value_counts(normalize=True)

False    0.96
True     0.04
Name: outcome_changed, dtype: float64

Here, we see that there was a change in outcome for just two of the proposals (4%). [This proposal](https://snapshot.org/#/lido-snapshot.eth/proposal/0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b) to [expand the oracle set and quorum](https://mainnet.lido.fi/#/lido-dao/0x442af784a788a5bd6f42a01ebe9f287a871243fb/) of Lido DAO oracle node operators was passed 16/12/22 and onboarded 4 new oracle node operators and set the quorum to 5/9 reports from oracles being identical to be accepted by the protocol.

Had whales not intervened, Option 1 to onboard 6 additional oracles and set the quorum to 6/11 would've passed.

In [17]:
propsal_choices = plutocracy_report_data['Lido'][plutocracy_report_data['Lido']['proposal_id'] == '0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Lido"].loc["0xcbf5343"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Lido'][plutocracy_report_data['Lido']['proposal_id'] == '0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,"1. For 6 new oracles, 6/11 set","2. For 4 new oracles, 5/9 set",3. Against
Scores,96886.616711,5.272470e+07,4778.954352
Score Differeces,92934.922644,5.272312e+07,4535.772055


Here we see that whale participation was consistent for all choices for this proposal. In fact, ~99.982% of voting power allocated to this proposal came from whales that voted for each of the outcomes. The second outcome alone had ~99.980% of the voting power allocated to this proposal, so obviously, its removal would make a huge impression on the outcome.

Indeed, the voting power would be comparatively low without whale intervention (if this were the case, only ~9277.7397 LDO would be allocated to this proposal). However the fact that after doing so additional oracles would still be implemented anyway, where the only other alternative would be to not implement any more oracles, shows that the community that participated in this proposal still saw it important to add more oracles to further promote decentralisation of oracles which [report chain state from the beacon chain to the execution layer](https://docs.lido.fi/guides/oracle-operator-manual) of ETH 2.0 as it pertains to the operation of Lido.

In conclusion of this analysis of this proposal, we can say that whales and average token holders were moving in the same direction, but perhaps taking different paths. It will be interesting to see why whales opted to implement fewer oracles than what the majority of average token holders chose, which may require a more in-depth report based on the [forum discussions](https://research.lido.fi/t/expansion-of-lidos-ethereum-oracle-set/2836) on the proposal.

---
Now we take a look at Frax.

In [18]:
score_diffrences_dfs["Frax"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0x1c844e1,weighted,"['For', 'Against']","[1432.8549342407946, 0]",2594.203582734,False,For
0x1127ca8,weighted,"['For', 'Against']","[4485.967956881548, 0.0]",7397.700806661,False,For
...,...,...,...,...,...,...
0x40e711b,weighted,"['For', 'Against']","[32776757.693645008, 0]",39954420.724392056,False,For
0x68c141f,weighted,"['For', 'Against']","[32776774.671120502, 0.0]",40485623.779861562,False,For


In [19]:
score_diffrences_dfs["Frax"]["outcome_changed"].value_counts(normalize=True)

False    0.98
True     0.02
Name: outcome_changed, dtype: float64

Whilst enormous voting power was filtered out from all the proposals listed, just one had their outcomes changed (2%).

If we look at the proposal which attracted the most voting power, [this proposal](https://snapshot.org/#/frax.eth/proposal/0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a) to renew the grant for Flywheel for 2023 (in the amount of approx $214.2k, split 50/50 in FXS and FRAX) for the promotion of "the Frax ecosystem and producing high-quality DeFi content", we see that just over 50% of the voting power was removed from this proposal after filtering out whales (approx 56.7%).

Though, because the opposition voting power for this proposal was a paltry 1E-5% of the vote, filtering out whales did not change the outcome. This was not a controversial outcome to this proposal after observing the number of voting participants (FXS token holders who participated in this proposal).

Indeed, Frax's [forum post](https://gov.frax.finance/t/fip-178-flywheel-grant-renewal-for-2023/2068) regarding this proposal had mostly positive things to say about Flywheel, which started out as a podcast promoting Frax in the wake of the collapse of Terra/Luna. Stating that since their last grant given to Flywheel in 2022, their social following has increased by 2770% on Twitter and 3140% on Youtube. They also serve as an educating branch for Frax, organizing meetups and also "attracted builders from across the ecosystem".

What's interesting about this proposal is that whale participation contributed to just over 56.7% of the voting power for this proposal. This is probably the most fairly represented outcome looked at so far, with the outcome being clearly popular with whales and the average voter.


In [20]:
propsal_choices = plutocracy_report_data['Frax'][plutocracy_report_data['Frax']['proposal_id'] == '0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Frax"].loc["0xece8d5b"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Frax'][plutocracy_report_data['Frax']['proposal_id'] == '0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,For,Against
Scores,1.172897e+07,131.88977
Score Differeces,9.238258e+06,0.00000


---
If we now look at Decentraland, we observe that there is quite a bad case of concentration of votes in the hands of plutocrats.

In [21]:
score_diffrences_dfs["Decentraland"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0xdbe9b2d,single-choice,"['yes', 'no']","[4002000.0, 0]",4018284.304020843,False,yes
0x7a6a819,single-choice,"['yes', 'no']","[4260611.0495, 9364.50282589451]",4271209.566700251,False,yes
...,...,...,...,...,...,...
0x4d2a825,single-choice,"['yes', 'no']","[0.0, 1650987.4791028618]",1724953.034168607,False,no
0x6f5a95f,single-choice,"['Remove Matimio', 'Keep Matimio', 'Invalid qu...","[144330.58336402333, 0.0, 1965041.5197398742]",2143801.982962364,False,Invalid question/options


In [22]:
score_diffrences_dfs["Decentraland"]["outcome_changed"].value_counts(normalize=True)

False    0.76
True     0.24
Name: outcome_changed, dtype: float64

Here we see that just under a quarter of proposals' outcomes would change if whales were not involved in Decentraland governance.

For example, this proposal to set a duration period for the tenure of Decentraland DAO commitee members saw 99.9% of voting power being concentrated to whales, with 94.85% of propsal voting power being allocated to voting for the proposal to not pass.

In [23]:
propsal_choices = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Decentraland"].loc["0x7f6fed8"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Yes,No,Invalid question/options
Scores,168338.243827,1.394610e+06,0.025206
Score Differeces,166209.638630,1.392896e+06,0.000000


Obviously a case of centralized power voting for their own self interests, its not suprising that if whales were factored out of the proposal (which would've given existing and future committee members set terms, making the roles more democratic) the proposal would've passed, albeit, with very low voting power in comparison.

---
Going back to DeFi, lets now look at Curve Finance.

In [24]:
score_diffrences_dfs["Curve Finance"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0x0eb23ea,single-choice,"['Yes', 'No']","[23462.994316173717, 511330.4724932205]",574553.874246682,True,Yes
0x43c689e,single-choice,"['Do Nothing', '2Pool + current 3Pool', '2Pool...","[0.0, 559204.7021513436, 2510541.702086272]",4272657.271177988,False,2Pool + current 3Pool + 4Pool
...,...,...,...,...,...,...
0xd946d40,single-choice,"['会', '不会']","[-15380.210202214721, 0]",2056.290716726,False,会
0x745856d,single-choice,"['Sure, I am diamond hand', 'No, I am paper ha...","[21295.537735671936, 0]",21311.073192843,False,"Sure, I am diamond hand"


In [25]:
score_diffrences_dfs["Curve Finance"]["outcome_changed"].value_counts(normalize=True)

False    0.714286
True     0.285714
Name: outcome_changed, dtype: float64

In [26]:
propsal_choices = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Curve Finance"].loc["0x0eb23ea"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Yes,No
Scores,45352.592228,529201.282019
Score Differeces,23462.994316,511330.472493


In [27]:
score_diffrences_dfs["Radicle"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0xf95b0e0,single-choice,"['For', 'Against']","[2000000.0, 0]",4133689.798082654,False,For
0x646e927,single-choice,"['Yes', 'No', 'Abstain']","[3730867.698436455, 0, 0]",6163639.615623794,False,Yes
...,...,...,...,...,...,...
QmVypVPUP,single-choice,"['For', 'Against']","[1820370.422397018, 0.0]",1962803.146575897,False,For
QmbpshrWc,single-choice,"['Unpause swapping', 'Do nothing']","[949045.9988789248, 20708.27072226024]",1291030.699274630,False,Unpause swapping


In [28]:
score_diffrences_dfs["Radicle"]["outcome_changed"].value_counts(normalize=True)

False    0.818182
True     0.181818
Name: outcome_changed, dtype: float64

In [29]:
score_diffrences_dfs["Compound"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0xba069fb,single-choice,"['Yes', 'No']","[205387.6973835756, 0.0]",205482.098511252,False,Yes
bafkreiba,single-choice,"['Migrate to Commonwealth', 'Do Nothing']","[100044.20000815867, 0.0]",100096.776618262,False,Migrate to Commonwealth
bafkreiar,single-choice,"['5%', '4%', '3%', '2%', '1%', '0%', 'Abstain']","[151547.28029712665, 1.0000999999999998, 3.895...",151614.525233746,False,5%
bafkreibb,single-choice,"['Decrease RF for USDT & DAI', 'Increase RF fo...","[50015.58777338891, 151502.5046347422, 10.9855...",201588.163223679,True,Wait until Compound III


In [30]:
score_diffrences_dfs["Compound"]["outcome_changed"].value_counts(normalize=True)

False    0.75
True     0.25
Name: outcome_changed, dtype: float64